In [ ]:
# rahul patil 20230901013
import numpy as np
import os
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA 
from PIL import Image 

In [49]:
data_dir = r"C:\Users\user\OneDrive\Desktop\MPOX Dataset\Original Images\FOLDS\fold2\Train"
target_size = (224, 224) 
pca_components = 8

In [50]:
image_data = [] # List to store image data
labels = [] # List to store labels
class_names = sorted(os.listdir(data_dir)) # Get class names
label_encoder = LabelEncoder()
print("Class names: ", class_names)

Class names:  ['Chickenpox', 'Cowpox', 'HFMD', 'Healthy', 'Measles', 'Monkeypox']


In [51]:
def extract_histogram_features(img_array, num_bins=8):
    """Extracts histogram features from an image."""
    hist = np.histogram(img_array, bins=num_bins)[0] # Get histogram
    return hist.flatten()

In [ ]:
for class_index, class_name in enumerate(class_names): 
    class_path = os.path.join(data_dir, class_name) # Construct path to class
    if os.path.isdir(class_path): 
        for image_file in os.listdir(class_path): 
            image_path = os.path.join(class_path, image_file) # Construct image path
            try:
                img = Image.open(image_path).convert('RGB').resize(target_size) # Open, convert, resize
                img_array = np.array(img) / 255.0 # Convert to NumPy array

                gray_img = np.mean(img_array, axis=2)

                features = extract_histogram_features(gray_img)

                image_data.append(features) # Append features
                labels.append(class_name) # Append label

            except Exception as e:
                print(f"Error processing {image_path}: {e}")

In [53]:
# Convert image_data to a NumPy array
image_data = np.array(image_data)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(image_data, labels_encoded, test_size=0.2, random_state=42)

# Initialize scaler
scaler = StandardScaler().fit(X_train)

# Scale training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
pca = PCA(n_components=pca_components)
X_train_pca = pca.fit_transform(X_train_scaled) 
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
model = svm.SVC(kernel='linear', C=1).fit(X_train_pca, y_train) 
y_pred = model.predict(X_test_pca) 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.28431372549019607


In [ ]:
#predictions for test image
test_image_path = r"C:\Users\user\OneDrive\Desktop\MPOX Dataset\Original Images\FOLDS\fold2\Test\HFMD\HFMD_91_01.jpg"
img = Image.open(test_image_path).convert('RGB').resize(target_size) # Open, convert, resize
img_array = np.array(img) / 255.0 
gray_img = np.mean(img_array, axis=2) 
features = extract_histogram_features(gray_img) 

test_image_scaled = scaler.transform([features]) 
test_image_pca = pca.transform(test_image_scaled) 

predicted_class_encoded = model.predict(test_image_pca)[0]
predicted_class = label_encoder.inverse_transform([predicted_class_encoded])[0]
print(f"Predicted class: {predicted_class}")


Predicted class: Monkeypox
